In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
characters_info = pd.read_csv('game_data/Character.Info2.csv').filter(['ID', 'RaceID', 'Speed']).drop_duplicates()
tmp = pd.read_csv('game_data/Character.Levels.Stats.csv')
characters_info = pd.merge(characters_info, tmp, on='ID')
tmp = pd.read_csv('game_data/Character.Levels.Info.csv').filter(['Level', 'RequiredTokens'])
characters_info = pd.merge(characters_info, tmp, on='Level')
characters_info['nft'] = characters_info['ID'].str.startswith('NFT_')
tmp = pd.read_csv('game_data/PricePool.csv')
tmp1 = tmp[(tmp['ID'].str.startswith('price_ch_upgrade_lvl_')) & (tmp['ItemID']=='Shards')]
tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
tmp1['Level'] = tmp1['Level'].astype(int)
tmp1['nft'] = False
tmp1 = tmp1.rename(columns={'Amount': 'Shards'})
tmp1['Crown'] = 0
tmp1 = tmp1.filter(['Level', 'Shards', 'Crown', 'nft'])
tmp2 = tmp[tmp['ID'].str.startswith('price_ch_nft_upgrade_lvl_')]
tmp2['Level'] = tmp2['ID'].str.extract('(\d+)')
tmp2['Level'] = tmp2['Level'].astype(int)
tmp2 = tmp2.filter(['Level', 'ItemID', 'Amount'])
tmp2 = tmp2.pivot(index='Level', columns='ItemID', values='Amount').reset_index()
tmp2['nft'] = True
tmp = pd.concat([tmp1, tmp2])
characters_info = pd.merge(characters_info, tmp, on=['Level', 'nft'])
characters_upgrade = characters_info.filter(['Level', 'RequiredTokens', 'Shards', 'Crown', 'nft']).rename(columns={'RequiredTokens': 'Tokens'}).drop_duplicates()
characters_info = characters_info.drop(columns=['RequiredTokens', 'Shards', 'Crown'])
characters_upgrade.to_csv('processed_game_data/characters_upgrade.csv', index=False)
characters_info = characters_info[~characters_info['ID'].isin(['NFT_Shark_Bronze', 'NFT_Shark_Silver'])]
def foo(row):
    names = {
        'Ape': 'Rocky',
        'Bear': 'Bonnie',
        'Cat': 'Cranky',
        'Tiger': 'Alex',
        'Croco': 'Aldo',
        'Rabbit': 'Lola',
        'Snail': 'Zoom',
        'Shark': 'Finn',
        'Squirrel': 'Zayn',
        'Bull': 'Brutus',
        'Chicken': 'Henley'
    }
    if row['ID'].startswith('FREE_'):
        return names[row['RaceID']]
    if row['ID'].startswith('NFT_Ape'):
        return row['ID'].split('_')[-1] + ' Genesis Ape'
    if row['ID'].startswith('NFT_Shark'):
        return row['ID'].split('_')[-1] + ' Space Shark'
    return np.nan
characters_info['ID'] = characters_info.apply(foo, axis=1)
characters_info = characters_info.rename(columns={'ID': 'Name', 'RaceID': 'Race'})
characters_info.to_csv('processed_game_data/characters_info.csv', index=False)

<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_21339/3255579976.py:9: SyntaxWarning: invalid escape sequence '\d'
  tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/3255579976.py:16: SyntaxWarning: invalid escape sequence '\d'
  tmp2['Level'] = tmp2['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/3255579976.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/3255579976.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [3]:
def fix_modifier_weapon_reloadTime_mul(value):
    if value < 1:
        return value
    return 2-value
weaponData = pd.read_csv('game_data/Weapon.Data.csv')
weaponInfo = pd.read_csv('game_data/Weapon.Info.csv')
weaponInfo_dict = {}
for weapon_type in weaponInfo['WeaponType'].unique():
    weaponInfo_dict[weapon_type] = {}
    for _, row in weaponInfo[weaponInfo['WeaponType']==weapon_type].iterrows():
        weaponInfo_dict[weapon_type][row['StatType']] = row['StatValue']
weaponInfo = pd.DataFrame.from_dict(weaponInfo_dict, orient='index').reset_index().rename(columns={'index': 'WeaponType'})
weaponInfo.to_csv('processed_game_data/weapons_params.csv', index=False)
weaponInfo = weaponInfo.drop(columns=weaponInfo.columns[weaponInfo.isna().any()].tolist())
weaponData = weaponData.filter(['ID', 'WeaponType', 'CollectionID', 'StarLevel', 'IsNFT', 'DamageIncrease'])
weaponInfo = pd.merge(weaponInfo, weaponData, on='WeaponType').drop(columns=['WeaponType'])
weaponInfo['level'] = 1
for lvl in range(2, 21):
    tmp = weaponInfo[weaponInfo['level']==1]
    tmp['level'] = lvl
    tmp['weapon_damage'] = tmp.apply(lambda row: float(row['weapon_damage']) + float(row['DamageIncrease'])*(lvl-1), axis=1)
    weaponInfo = pd.concat([weaponInfo, tmp])
weaponInfo = weaponInfo.drop(columns=['DamageIncrease'])
weaponInfo.to_csv('processed_game_data/weapons_info.csv', index=False)
weaponInfo = pd.read_csv('processed_game_data/weapons_info.csv')
tmp = pd.read_csv('game_data/Weapon.Data.csv').filter(['ID', 'StatType', 'StatValue'])
tmp['StarLevel'] = tmp['ID'].str.extract('(\d+)')
tmp['StarLevel'] = tmp['StarLevel'].astype(int)
tmp = tmp.sort_values('StarLevel')
for i, row in tmp.iterrows():
    statType = row['StatType']
    statValue = row['StatValue']
    starLevel = row['StarLevel']
    id = '_'.join(row['ID'].split('_')[:-1])
    if statType == 'weapon_damage':
        weaponInfo['weapon_damage'] = weaponInfo.apply(lambda x: x['weapon_damage'] + statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_damage'], axis=1)
    if statType == 'weapon_projectile_range':
        weaponInfo['weapon_projectile_range'] = weaponInfo.apply(lambda x: x['weapon_projectile_range'] + statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_projectile_range'], axis=1)
    if statType == 'modifier_weapon_damage_mul':
        weaponInfo['weapon_damage'] = weaponInfo.apply(lambda x: x['weapon_damage'] * statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_damage'], axis=1)
    if statType == 'modifier_weapon_reloadTime_mul':
        weaponInfo['weapon_reloadTime'] = weaponInfo.apply(lambda x: x['weapon_reloadTime'] * fix_modifier_weapon_reloadTime_mul(statValue) if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_reloadTime'], axis=1)
    if statType == 'modifier_weapon_projectile_speed_mul':
        weaponInfo['weapon_projectile_speed'] = weaponInfo.apply(lambda x: x['weapon_projectile_speed'] * statValue if x['StarLevel']>=starLevel and '_'.join(x['ID'].split('_')[:-1])==id else x['weapon_projectile_speed'], axis=1)
names = {
    'AssaultGun Default': 'Blaster',
    'Shotgun Default': 'Boomstick',
    'RailGun Default': 'Railgun',
    'MortarGun Default': 'Mortar Gun',
    'TeslaGun Default': 'Tesla Gun',
    'AssaultGun Arsenal-X': 'Blaster-X',
    'Shotgun Arsenal-X': 'Boomstick-X',
    'RailGun Arsenal-X': 'Railgun-X',
    'MortarGun Arsenal-X': 'Mortar Gun X',
    'TeslaGun Arsenal-X': 'Tesla Gun X',
    'Hammer Mythical': 'Skybreaker',
    'RocketLauncher Mythical': 'Hellfire Launcher',
    'Flamer Mythical': 'Dragonbreath',
    'Minigun Mythical': 'Titan Shredder',
    'Spreadgun Mythical': 'Helios Cannon',
    'Crossbow Mythical': 'Wrath Of Artemis'
}
weaponInfo['name'] = weaponInfo['ID'].apply(lambda x: names[' '.join(x.split('_')[1:-1])])
weaponInfo = weaponInfo.drop(columns=['ID'])
weaponInfo.to_csv('processed_game_data/weapons_info.csv', index=False)

<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_21339/4222448104.py:27: SyntaxWarning: invalid escape sequence '\d'
  tmp['StarLevel'] = tmp['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/4222448104.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['level'] = lvl
/tmp/ipykernel_21339/4222448104.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['weapon_damage'] = tmp.apply(lambda row: float(row['weapon_damage']) + float(row['

In [4]:
weapons_upgrade = pd.read_csv('game_data/Weapon.Levels.csv')
tmp = pd.read_csv('game_data/PricePool.csv')
tmp1 = tmp[(tmp['ID'].str.startswith('price_wp_upgrade_lvl_')) & (tmp['ItemID']=='Shards')]
tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
tmp1['Level'] = tmp1['Level'].astype(int)
tmp1['nft'] = False
tmp1 = tmp1.rename(columns={'Amount': 'Shards'})
tmp1['Crown'] = 0
tmp1 = tmp1.filter(['Level', 'Shards', 'Crown', 'nft'])
tmp2 = tmp[tmp['ID'].str.startswith('price_wp_nft_upgrade_lvl_')]
tmp2['Level'] = tmp2['ID'].str.extract('(\d+)')
tmp2['Level'] = tmp2['Level'].astype(int)
tmp2 = tmp2.filter(['Level', 'ItemID', 'Amount'])
tmp2 = tmp2.pivot(index='Level', columns='ItemID', values='Amount').reset_index()
tmp2['nft'] = True
tmp = pd.concat([tmp1, tmp2])
weapons_upgrade = pd.merge(weapons_upgrade, tmp, on='Level')
weapons_upgrade = weapons_upgrade.rename(columns={'XPNeeded': 'Tokens'})
weapons_upgrade = weapons_upgrade.filter(['Level', 'Tokens', 'Shards', 'Crown', 'nft'])
weapons_upgrade = weapons_upgrade.drop_duplicates().sort_values(['nft', 'Level'])
weapons_upgrade.to_csv('processed_game_data/weapons_upgrade.csv', index=False)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_21339/266754270.py:4: SyntaxWarning: invalid escape sequence '\d'
  tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/266754270.py:11: SyntaxWarning: invalid escape sequence '\d'
  tmp2['Level'] = tmp2['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/266754270.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['Level'] = tmp1['ID'].str.extract('(\d+)')
/tmp/ipykernel_21339/266754270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [5]:
characters_info_dict = {'free': {}, 'nft': {}}
for _, row in characters_info.iterrows():
    key = 'free' if not row['nft'] else 'nft'
    d = {
        'race': row['Race'],
        'speed': row['Speed'],
        'levels': {}
    }
    if row['Name'] not in characters_info_dict['nft']:
        characters_info_dict[key][row['Name']] = d
    characters_info_dict[key][row['Name']]['levels'][row['Level']] = {
        'health': row['Health'],
        'shield': row['Shield'],
        'talentPoints': row['TalentPoints'],
        'skillLevel': row['SkillLevel'],
        'crownEarnRate': row['CrownEarnRate']
    }
with open('../public/data/characters_info.json', 'w') as f:
    json.dump(characters_info_dict, f, indent=4)

In [6]:
characters_upgrade_dict = {'free': {}, 'nft': {}}
for _, row in characters_upgrade.iterrows():
    key = 'free' if not row['nft'] else 'nft'
    characters_upgrade_dict[key][row['Level']] = {
        'tokens': row['Tokens'],
        'shards': row['Shards'],
        'crown': row['Crown']
    }
with open('../public/data/characters_upgrade.json', 'w') as f:
    json.dump(characters_upgrade_dict, f, indent=4)

In [7]:
weaponInfo.columns

Index(['weapon_class', 'weapon_damage', 'weapon_cooldownAfterLastBullet',
       'weapon_reloadTime', 'weapon_ammo', 'weapon_projectile_speed',
       'weapon_projectile_range', 'weapon_projectile_spawn_delay_after_shot',
       'CollectionID', 'StarLevel', 'IsNFT', 'level', 'name'],
      dtype='object')

In [8]:
weapon_info_dict = {}
for _, row in weaponInfo.iterrows():
    collection = row['CollectionID']
    name = row['name']
    starLevel = row['StarLevel']
    level = row['level']
    if collection not in weapon_info_dict:
        weapon_info_dict[collection] = {}
    if name not in weapon_info_dict[collection]:
        weapon_info_dict[collection][name] = {
            'class': row['weapon_class'],
            'isNFT': row['IsNFT'],
            'starLevels': {}
        }
    if starLevel not in weapon_info_dict[collection][name]['starLevels']:
        weapon_info_dict[collection][name]['starLevels'][starLevel] = {}
    weapon_info_dict[collection][name]['starLevels'][starLevel][level] = {
        'damage': row['weapon_damage'],
        'cooldownAfterLastBullet' : row['weapon_cooldownAfterLastBullet'],
        'reloadTime': row['weapon_reloadTime'],
        'ammo': row['weapon_ammo'],
        'projectile_speed': row['weapon_projectile_speed'],
        'projectile_range': row['weapon_projectile_range'],
        'projectile_spawn_delay_after_shot': row['weapon_projectile_spawn_delay_after_shot']
    }
with open('../public/data/weapons_info.json', 'w') as f:
    json.dump(weapon_info_dict, f, indent=4)
        

In [9]:
weapons_upgrade_dict = {'free': {}, 'nft': {}}
for _, row in weapons_upgrade.iterrows():
    key = 'free' if not row['nft'] else 'nft'
    weapons_upgrade_dict[key][row['Level']] = {
        'tokens': row['Tokens'],
        'shards': row['Shards'],
        'crown': row['Crown']
    }
with open('../public/data/weapons_upgrade.json', 'w') as f:
    json.dump(weapons_upgrade_dict, f, indent=4)